# Preparation and Imports


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import glob
import os
from common import *
import json
from datetime import datetime
from matplotlib import ticker
from matplotlib.colors import LogNorm, Normalize
import seaborn as sns
import numpy as np
import jenkspy

#plt.style.use('thesis.mplstyle')


In [2]:
attribute_combinations=pd.read_csv('crawl_results_Donothing/attribute_combinations_entropyJoint.csv')
attribute_combinations

,domain,destinationHost,attribute_combinations,entropy
0,009.am,009.xn--p1ai,navigator.useragent,0.564534
1,009.am,google-analytics.com,"navigator.language, screen.colordepth, screen....",0.796308
2,009.am,yandex.com,"navigator.language, screen.width, screen.heigh...",0.738580
3,00web.net,00web.net,navigator.useragent,0.564534
4,00web.net,doubleclick.net,"navigator.useragent, history.length, screen.he...",0.763784
...,...,...,...,...
269806,zzz.com.ua,google-analytics.com,"navigator.language, screen.colordepth, screen....",0.796308
269807,zzz.com.ua,undefined.com.ua,navigator.useragent,0.564534
269808,zzz.com.ua,zzz.com.ua,navigator.useragent,0.564534
269809,zzzfun.com,51.la,"screen.width, screen.height, navigator.languag...",0.653779


# Load and Prepare Data

## Website Lists

### Tranco


In [3]:
tranco = pd.read_csv('tranco_top-1m_N7QVW_2022-10-07.csv',
                     names=['pagerank', 'domain']).iloc[0:500000, :]
#tranco['rank_group'] = pd.cut(tranco.pagerank, bins=rank_group_bins)
tranco


,pagerank,domain
0,1,google.com
1,2,gtld-servers.net
2,3,youtube.com
3,4,facebook.com
4,5,microsoft.com
...,...,...
499995,499996,reitbuch.com
499996,499997,matrixcos.com
499997,499998,soodiran.com
499998,499999,onepunchman-anime.net


### sample example


### Donothing crawl


In [4]:
statusReport= pd.read_csv('crawl_results_Donothing/raw_status.csv')
statusReport

,domain,start,end,hasErrors,error,originalFile,reachedDomain
0,gtld-servers.net,2023-03-31T11:48:04.212Z,2023-03-31T11:48:10.081Z,True,unreachable,results.json_00000000-00000009.json,NaN
1,akamaiedge.net,2023-03-31T11:48:04.214Z,2023-03-31T11:48:10.367Z,True,unreachable,results.json_00000000-00000009.json,NaN
2,google.com,2023-03-31T11:48:04.196Z,2023-03-31T11:49:17.346Z,False,Successful,results.json_00000000-00000009.json,google.com
3,apple.com,2023-03-31T11:48:04.217Z,2023-03-31T11:49:19.626Z,False,Successful,results.json_00000000-00000009.json,apple.com
4,microsoft.com,2023-03-31T11:48:04.213Z,2023-03-31T11:49:21.187Z,False,Successful,results.json_00000000-00000009.json,microsoft.com
...,...,...,...,...,...,...,...
100082,easternct.edu,2023-04-17T09:17:57.572Z,2023-04-17T09:19:22.811Z,False,Successful,results.json_00089810-00089819.json,easternct.edu
100083,kyoto.travel,2023-04-17T09:17:55.481Z,2023-04-17T09:19:29.740Z,False,Successful,results.json_00089810-00089819.json,kyoto.travel
100084,crosswordbuzz.com,2023-04-17T09:10:33.509Z,2023-04-17T09:19:33.821Z,True,unspecified error,results.json_00089810-00089819.json,NaN
100085,williamhill.it,2023-04-17T09:19:33.822Z,2023-04-17T09:19:41.801Z,True,unreachable,results.json_00089810-00089819.json,NaN


In [5]:

flowReport = pd.read_csv('crawl_results_Donothing/raw_flowreport.csv')
flowReport.drop_duplicates(ignore_index=True)

,domain,consentMode,page,pageHost,frameHost,destinationHost,source,sourceHost,sink,timeStamp,sinkHost
0,google.com,doNothing,2,google.com,smallbusiness.withgoogle.com,google-analytics.com,Navigator.language,googletagmanager.com,navigator.sendBeacon(url),2023-03-31 11:48:43.130000+00:00,googletagmanager.com
1,google.com,doNothing,2,google.com,smallbusiness.withgoogle.com,google-analytics.com,Screen.width,googletagmanager.com,navigator.sendBeacon(url),2023-03-31 11:48:43.130000+00:00,googletagmanager.com
2,google.com,doNothing,2,google.com,smallbusiness.withgoogle.com,google-analytics.com,Screen.height,googletagmanager.com,navigator.sendBeacon(url),2023-03-31 11:48:43.130000+00:00,googletagmanager.com
3,google.com,doNothing,2,google.com,smallbusiness.withgoogle.com,smallbusiness.withgoogle.com,Navigator.userAgent,google-analytics.com,document.cookie,2023-03-31 11:48:43.691000+00:00,google-analytics.com
4,google.com,doNothing,2,google.com,smallbusiness.withgoogle.com,google-analytics.com,Navigator.language,google-analytics.com,XMLHttpRequest.open(url),2023-03-31 11:48:43.697000+00:00,google-analytics.com
...,...,...,...,...,...,...,...,...,...,...,...
7554431,kyoto.travel,doNothing,4,kyoto.travel,kyoto.travel,google.com,Screen.width,googletagmanager.com,navigator.sendBeacon(url),2023-04-17 09:19:26.284000+00:00,googletagmanager.com
7554432,kyoto.travel,doNothing,4,kyoto.travel,kyoto.travel,google.com,Screen.height,googletagmanager.com,navigator.sendBeacon(url),2023-04-17 09:19:26.284000+00:00,googletagmanager.com
7554433,kyoto.travel,doNothing,4,kyoto.travel,kyoto.travel,google.com,Navigator.language,googletagmanager.com,navigator.sendBeacon(url),2023-04-17 09:19:26.287000+00:00,googletagmanager.com
7554434,kyoto.travel,doNothing,4,kyoto.travel,kyoto.travel,google.com,Screen.width,googletagmanager.com,navigator.sendBeacon(url),2023-04-17 09:19:26.287000+00:00,googletagmanager.com


In [6]:

#transform the date format
statusReport.start = pd.to_datetime(statusReport.start)
statusReport.end = pd.to_datetime(statusReport.end)

#add column named status and filter out successfull from unsuccessfull crawls and specify the crawler 
statusReport['status'] = 'error in Donothing crawl'
statusReport.loc[~(statusReport.hasErrors), 'status'] = 'successful'

# add comlumn simplified error
statusReport['statusSimplified'] = statusReport.error
statusReport.loc[statusReport.error == 'Successful', 'statusSimplified'] = 'successful'
statusReport.loc[statusReport.error == 'argtype error', 'statusSimplified'] = 'other crawler error'
statusReport.loc[statusReport.error == 'unspecified error', 'statusSimplified'] = 'other crawler error'
statusReport.loc[statusReport.error == 'domain skipped', 'statusSimplified'] = 'other crawler error'



# Get the tld of the crawled website
statusReport['tld'] = statusReport.domain.str.split('.').str[-1]

statusReport




,domain,start,end,hasErrors,error,originalFile,reachedDomain,status,statusSimplified,tld
0,gtld-servers.net,2023-03-31 11:48:04.212000+00:00,2023-03-31 11:48:10.081000+00:00,True,unreachable,results.json_00000000-00000009.json,NaN,error in Donothing crawl,unreachable,net
1,akamaiedge.net,2023-03-31 11:48:04.214000+00:00,2023-03-31 11:48:10.367000+00:00,True,unreachable,results.json_00000000-00000009.json,NaN,error in Donothing crawl,unreachable,net
2,google.com,2023-03-31 11:48:04.196000+00:00,2023-03-31 11:49:17.346000+00:00,False,Successful,results.json_00000000-00000009.json,google.com,successful,successful,com
3,apple.com,2023-03-31 11:48:04.217000+00:00,2023-03-31 11:49:19.626000+00:00,False,Successful,results.json_00000000-00000009.json,apple.com,successful,successful,com
4,microsoft.com,2023-03-31 11:48:04.213000+00:00,2023-03-31 11:49:21.187000+00:00,False,Successful,results.json_00000000-00000009.json,microsoft.com,successful,successful,com
...,...,...,...,...,...,...,...,...,...,...
100082,easternct.edu,2023-04-17 09:17:57.572000+00:00,2023-04-17 09:19:22.811000+00:00,False,Successful,results.json_00089810-00089819.json,easternct.edu,successful,successful,edu
100083,kyoto.travel,2023-04-17 09:17:55.481000+00:00,2023-04-17 09:19:29.740000+00:00,False,Successful,results.json_00089810-00089819.json,kyoto.travel,successful,successful,travel
100084,crosswordbuzz.com,2023-04-17 09:10:33.509000+00:00,2023-04-17 09:19:33.821000+00:00,True,unspecified error,results.json_00089810-00089819.json,NaN,error in Donothing crawl,other crawler error,com
100085,williamhill.it,2023-04-17 09:19:33.822000+00:00,2023-04-17 09:19:41.801000+00:00,True,unreachable,results.json_00089810-00089819.json,NaN,error in Donothing crawl,unreachable,it


In [7]:
reach=statusReport.pivot_table(values='domain',index='error',aggfunc='count',margins=True)

#saveLatexTable(reach, 'crawled_successfully.tex')

reach

,domain
error,
Successful,80671
Unkonw error in processing,2
argtype error,163
domain skipped,74
timeout error,1512
unreachable,15324
unspecified error,2341
All,100087


In [8]:
statusReport.loc[ (statusReport.error != 'Successful'),'error'] = 'scanning error'
statusReport.loc[(statusReport.error != 'Successful'),'statusSimplified'] = 'scanning error'

reach=statusReport.pivot_table(values='domain',index='error',aggfunc='count',margins=True)


#saveLatexTable(reach, 'crawled_successfully_error.tex')

reach

,domain
error,
Successful,80671
scanning error,19416
All,100087


## Flows


In [9]:

#Remove malformed Destinations
flowReport.drop(flowReport.loc[flowReport.destinationHost.str.startswith('Malform') |flowReport.destinationHost.str.startswith('ERROR') |flowReport.destinationHost.str.startswith('malform') | flowReport.destinationHost.isnull()].index, inplace=True)

flowReport.drop(flowReport.loc[flowReport.destinationHost.str.startswith('UNKOWN') | flowReport.destinationHost.isnull()].index, inplace=True)

# add filtering ERROR.null.domain


# sink to source pairs
flowReport['transmission'] = flowReport.source + \
    ':' + flowReport.destinationHost

#flowReport.to_csv('flow_processsed.csv', index=False)

In [10]:
flowReport.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7554071 entries, 0 to 7554435
Data columns (total 12 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   domain           object
 1   consentMode      object
 2   page             int64 
 3   pageHost         object
 4   frameHost        object
 5   destinationHost  object
 6   source           object
 7   sourceHost       object
 8   sink             object
 9   timeStamp        object
 10  sinkHost         object
 11  transmission     object
dtypes: int64(1), object(11)
memory usage: 749.2+ MB


In [ ]:
# mark as unknown if destination host is null
flowReport.loc[flowReport.destinationHost.isna(),
               'destinationHost'] = 'UNKNOWN'
flowReport.drop(flowReport.loc[flowReport.destinationHost.str.startswith('UNKOWN') | flowReport.destinationHost.isnull()].index, inplace=True)


#filter out first party
flowReport['destinationSimplified'] = flowReport.destinationHost
flowReport.loc[flowReport.destinationHost ==
               flowReport.pageHost, 'destinationSimplified'] = 'first-party'


#filter out first party in the sinkHOST
flowReport['HostSinkSimplified'] = flowReport.sinkHost
flowReport.loc[flowReport.sinkHost ==
               flowReport.pageHost, 'HostSinkSimplified'] = 'first-party'

flowReport.loc[(flowReport.sinkHost !=
               flowReport.pageHost) , 'HostSinkSimplified'] ='third-party'

flowReport.to_csv('flow_processsed.csv', index=False)

In [ ]:
flowReport.pivot_table(values='transmission',index='source',aggfunc='count',margins=True)


In [ ]:

flowReport = pd.read_csv('flow_processsed.csv')

flowReport

In [ ]:
attribute_combinations = flowReport.groupby(['domain','destinationHost'])['source'].apply(lambda x: list(x.unique())).reset_index(name='attribute_combinations')
attribute_combinations



# Key Numbers


### Start, End, Duration


In [ ]:
from datetime import datetime
crawlStart =  datetime.fromisoformat(statusReport.start.min().rstrip('Z'))
crawlEnd = datetime.fromisoformat( statusReport.loc[~(statusReport.end.isna()), 'end'].max().rstrip('Z'))
crawlDuration = crawlEnd-crawlStart
perDomainTime = crawlDuration/100000


crawlEndTime = statusReport.end.max() 

print('Crawl Start     ', crawlStart)
print('Crawl End       ', crawlEnd)
print('Crawl Duration  ', crawlDuration)




### Successful, Unsuccessful


In [ ]:
totalDomains = statusReport.shape[0]
successfulDomains = statusReport[~(statusReport.hasErrors)].shape[0]
errorDomains = statusReport[statusReport.hasErrors].shape[0]

print('Total Domain Count       ', totalDomains)
print('Successful Domain Count  ', successfulDomains)
print('Error Domain Count       ', errorDomains)


In [ ]:
crawledDomainsAtRanks = statusReport.pivot_table(
    index='rank_group', aggfunc={'domain': 'count'}).domain
crawledDomainsAtRanks


In [ ]:
successfulDomainsAtRanks = statusReport[~(statusReport.hasErrors)]\
    .pivot_table(index='rank_group', aggfunc={'domain': 'count'})\
    .domain
successfulDomainsAtRanks


In [ ]:
statusReport

### Page Counts


In [ ]:
pagesPerDomain = flowReport.page.max() + 1
successfulPagesPerConsentMode = successfulDomains * pagesPerDomain
successfulPages = successfulPagesPerConsentMode * 1

print('Pages per Domain                   ', pagesPerDomain)
print('Successful Domain Count            ', successfulDomains)
print('Successful Pages per consent mode  ', successfulPagesPerConsentMode)
print('Successful Pages Overall           ', successfulPages)


## Redirects


In [ ]:
statusReport.pivot_table(index='error', columns=(
    statusReport.reachedDomain == statusReport.domain), aggfunc={'domain': 'count'})


In [ ]:
redirects = statusReport.loc[~(
    statusReport.reachedDomain == statusReport.domain) & ~statusReport.hasErrors, :]
redirects.to_csv('redirects.csv')
redirects


# Classification Joint_entropy


In [ ]:
#Copy of sum_entrpy-unique_combi.csv
Entropy=pd.read_csv('entropy.csv')
# Convert attribute combinations in Entropy DataFrame to frozenset as well

Entropy


In [ ]:
import pandas as pd

# Assuming Entropy DataFrame has columns: 'Vectors', 'joint_H', 'joint_H_n', 'anon_set'
# Assuming ranges and levels are defined as mentioned in the question

# Create a new column in Entropy DataFrame with categories based on 'joint_H_n' ranges
Entropy['level'] = pd.cut(Entropy['joint_H_n'], bins=[0.0,
 0.154098186,
 0.442675059,
 0.596961491,
 0.705353706,
 0.832438884,
 0.988612233], labels=['Negligible1','Negligible2', 'Low', 'Medium', 'High', 'very high'])

# Calculate the average anon_set per group
average_anon_set_per_group = Entropy.groupby('level')['anon_set'].mean().reset_index()

print(average_anon_set_per_group)


In [ ]:



# Plot joint_H_n against anon_set with regression line
plt.figure(figsize=(8, 6))
sns.scatterplot(x='joint_H_n', y='anon_set', data=Entropy, hue='level', palette='viridis')

# Add regression line
sns.regplot(x='joint_H_n', y='anon_set', data=Entropy, scatter=False, color='black')

plt.title('joint_H_n vs anon_set with Regression Line')
plt.xlabel('joint_H_n')
plt.ylabel('anon_set')
plt.legend(title='Level')

plt.grid(True)
plt.show()



In [ ]:


levels = ['Negligible', 'Low', 'Medium', 'High', 'very high']
line_colors = ['purple', 'green', 'yellow', 'orange', 'red']

# Filter out zero and negative values in 'anon_set'
filtered_entropy = Entropy[Entropy['anon_set'] > 0]

# Map the levels to a new merged 'Negligible' level
filtered_entropy['level_merged'] = filtered_entropy['level'].apply(lambda x: 'Negligible' if 'Negligible' in x else x)

# Plot joint_H_n against anon_set with regression line
plt.figure(figsize=(8, 6))
sns.scatterplot(x='joint_H_n', y='anon_set', data=filtered_entropy, hue='level_merged', palette=line_colors)

# Calculate the coefficients of the least squares line (linear regression)
x = filtered_entropy['joint_H_n']
y = filtered_entropy['anon_set']
coefficients = np.polyfit(x, np.log(y), 1)
m, c = coefficients

# Plot the least squares line
plt.plot(x, np.exp(m * x + c), color='black', linestyle='-', label='Least Squares Line')

plt.title('joint_H_n vs anon_set with Least Squares Line')
plt.xlabel('joint_H_n')
plt.ylabel('anon_set')
plt.legend(title='Level')

plt.yscale('log')  # Set y-axis to logarithmic scale
plt.grid(True)
plt.show()



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

levels = ['Negligible', 'Low', 'Medium', 'High', 'very high']
line_colors = ['purple', 'green', 'yellow', 'orange', 'red']

# Filter out zero and negative values in 'anon_set'
filtered_entropy = Entropy[Entropy['anon_set'] > 0]

# Map the levels to a new merged 'Negligible' level
filtered_entropy['level_merged'] = filtered_entropy['level'].apply(lambda x: 'Negligible' if 'Negligible' in x else x)

# Create a figure and axes
plt.figure(figsize=(8, 6))

# Scatter plot with smaller dots
sns.scatterplot(x='joint_H_n', y='anon_set', data=filtered_entropy, color='black', s=10)  # Set the size using 's'

# Calculate the coefficients of the least squares line (linear regression)
x = filtered_entropy['joint_H_n']
y = filtered_entropy['anon_set']
coefficients = np.polyfit(x, np.log(y), 1)
m, c = coefficients

# Plot the least squares line
plt.plot(x, np.exp(m * x + c), color='black', linestyle='-', label='Least Squares Line')

# Define your ranges
ranges = [(0.0, 0.442675059), (0.442675059, 0.596961491), (0.596961491, 0.705353706), (0.705353706, 0.832438884), (0.832438884, 0.988612233)]

# Plot shaded regions with the same color as the key
for i, (r, color) in enumerate(zip(ranges, line_colors)):
    plt.gca().add_patch(plt.Rectangle((r[0], 0), r[1] - r[0], plt.gca().get_ylim()[1], color=color, alpha=0.2, label=levels[i]))

# Set labels and title
plt.xlabel('Normalized Joint Entropy')
plt.ylabel('anonymaty set')
plt.legend(loc='upper right')
plt.yscale('log')  # Set y-axis to logarithmic scale
plt.grid(True)
plt.show()



In [ ]:
import matplotlib.pyplot as plt

# Plotting the histogram
n, bins, patches = plt.hist(Entropy["joint_H_n"], bins=20, edgecolor='black')  # Adjust the number of bins as needed
plt.xlabel('Entropy', fontsize=10)
plt.ylabel('Frequency', fontsize=10)
plt.title('Count of distributed entropy per unique combinations of fingerprinting attributes', fontsize=12)

# Adding text on top of each peak
for i in range(len(patches)):
    x = bins[i] + (bins[i+1] - bins[i]) / 2  # Calculate x-coordinate for the text
    y = n[i]  # Get the height of the current peak
    plt.text(x, y, str(int(y)), ha='center', va='bottom', fontsize=8)

# Modify x-axis ticks to show bin numbers
plt.xticks(fontsize=8)
plt.yticks(fontsize=8)

# Display the histogram
plt.show()
plt.savefig('histogram_of_distributed_entropy_per_unique_combination.png', dpi=300)



In [ ]:
attribute_combinations = flowReport.groupby(['domain','destinationHost'])['source'].apply(lambda x: list(x.unique())).reset_index(name='attribute_combinations')
attribute_combinations

In [ ]:
attribute_combinations['attribute_combinations'] = attribute_combinations['attribute_combinations'].astype(str)
attribute_combinations['attribute_combinations'] = attribute_combinations['attribute_combinations'].str.replace('[\[\]]', '', regex=True)
attribute_combinations['attribute_combinations'] = attribute_combinations['attribute_combinations'].str.replace("'", '', regex=True)


# Convert the 'attribute_combinations' column in attribute_combinations DataFrame to lowercase
attribute_combinations['attribute_combinations'] = attribute_combinations['attribute_combinations'].str.lower()

# Convert the 'attribute_combinations' column in Entropy DataFrame to lowercase
Entropy['Vectors'] = Entropy['Vectors'].str.lower()

# Create a new column in attribute_combinations DataFrame to store the matched entropy values
attribute_combinations['entropy'] = attribute_combinations['attribute_combinations'].apply(
    lambda x: Entropy.loc[Entropy['Vectors'].apply(
        lambda y: set(map(str.strip, y.split('|'))) == set(map(str.strip, x.split(','))))
    ]["joint_H_n"].iloc[0] if not pd.isna(x) and len(Entropy.loc[Entropy['Vectors'].apply(
        lambda y: set(map(str.strip, y.split('|'))) == set(map(str.strip, x.split(','))))
    ]["joint_H_n"]) > 0 else None
)

# Fill the NaN values in the entropy column with 0
attribute_combinations['entropy'] = attribute_combinations['entropy'].fillna(0)



In [ ]:

# Save the modified DataFrame as a CSV file
attribute_combinations.to_csv('attribute_combinations_entropyJoint.csv', index=False)